In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

caminho_foto_objeto = 'caneca.jpeg'
caminho_novo_fundo = 'cr7teste.jpg'

foto_objeto_bgr = cv2.imread(caminho_foto_objeto)
foto_fundo_bgr = cv2.imread(caminho_novo_fundo)

if foto_objeto_bgr is None:
    print(f"Erro: Imagem não encontrada em '{caminho_foto_objeto}'. Verifique o nome do arquivo.")
elif foto_fundo_bgr is None:
    print(f"Erro: Imagem não encontrada em '{caminho_novo_fundo}'. Verifique o nome do arquivo.")
else:
    foto_objeto = cv2.cvtColor(foto_objeto_bgr, cv2.COLOR_BGR2RGB)
    foto_fundo = cv2.cvtColor(foto_fundo_bgr, cv2.COLOR_BGR2RGB)

    foto_fundo = cv2.resize(foto_fundo, (foto_objeto.shape[1], foto_objeto.shape[0]))

    gray = cv2.cvtColor(foto_objeto_bgr, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)

        mask_objeto = np.zeros(gray.shape, dtype=np.uint8)

        cv2.drawContours(mask_objeto, [largest_contour], -1, 255, -1)

        kernel = np.ones((5,5), np.uint8)
        mask_objeto = cv2.morphologyEx(mask_objeto, cv2.MORPH_CLOSE, kernel)
        mask_objeto = cv2.morphologyEx(mask_objeto, cv2.MORPH_OPEN, kernel)

    else:
        print("Nenhum contorno grande o suficiente foi encontrado. A máscara será preta.")
        mask_objeto = np.zeros(gray.shape, dtype=np.uint8)

    mask_objeto_rgb = cv2.cvtColor(mask_objeto, cv2.COLOR_GRAY2RGB)

    objeto_recortado = cv2.bitwise_and(foto_objeto, mask_objeto_rgb)

    fundo_furado = cv2.bitwise_and(foto_fundo, cv2.bitwise_not(mask_objeto_rgb))

    resultado_final = cv2.add(objeto_recortado, fundo_furado)

    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    axs[0].imshow(foto_objeto)
    axs[0].set_title('1. Foto Original')
    axs[0].axis('off')

    axs[1].imshow(mask_objeto, cmap='gray')
    axs[1].set_title('2. Máscara do Objeto (Recorte)')
    axs[1].axis('off')

    axs[2].imshow(resultado_final)
    axs[2].set_title('3. Resultado Final')
    axs[2].axis('off')

    plt.tight_layout()
    plt.show()